In [1]:
import os
import sys
import pandas as pd
from PIL import Image

sys.path.append('../src/')

Посмотрим на скачанные данные

In [2]:
RAW_CSV = "../data/raw/test-task/clusters.csv"                  
CLEANED_CSV = "../data/processed/clusters_cleaned.csv"
RAW_IMAGES = "../data/raw/test-task/clusters/"
CLEANED_IMAGES = "../data/processed/cleaned_clusters/"

In [3]:
dataframe = pd.read_csv(RAW_CSV)
print(dataframe.shape)
dataframe.head()

(1521, 3)


,Unnamed: 0,cluster_id,file_name
0,0,f75f7082d62a44e7bfd373532877c9a6,70d7af617866428ba9d49a77505b8de0.jpg
1,1,f75f7082d62a44e7bfd373532877c9a6,075eb928960e46e0ae058f3e3e2efd4f.jpg
2,2,f75f7082d62a44e7bfd373532877c9a6,81f97e936ce84a5fa1ccabff3ad75725.jpg
3,3,f75f7082d62a44e7bfd373532877c9a6,3cbe3279c96848aa8bf24cb54e9de861.jpg
4,4,f75f7082d62a44e7bfd373532877c9a6,376ba3f5dfc44bfa90a4d72df1d5000f.jpg


In [4]:
dataframe.drop(columns=dataframe.columns[0], axis=1, inplace=True)

В какой-то момент я осознал, что все изображения без лиц имеют одинаковое разрешение - (1080, 1920). Почистим датасет от них

In [5]:
true_ids = []

for path in os.listdir(RAW_IMAGES):
    full_path = RAW_IMAGES + '/' + path
    image = Image.open(full_path)
    if image.size != (1920, 1080) and image.size != (1080, 1920):
        true_ids.append(path)

In [6]:
cleaned_dataframe = dataframe[dataframe['file_name'].isin(true_ids)] 
print(cleaned_dataframe.shape)
cleaned_dataframe.head()

(557, 2)


,cluster_id,file_name
0,f75f7082d62a44e7bfd373532877c9a6,70d7af617866428ba9d49a77505b8de0.jpg
1,f75f7082d62a44e7bfd373532877c9a6,075eb928960e46e0ae058f3e3e2efd4f.jpg
2,f75f7082d62a44e7bfd373532877c9a6,81f97e936ce84a5fa1ccabff3ad75725.jpg
3,f75f7082d62a44e7bfd373532877c9a6,3cbe3279c96848aa8bf24cb54e9de861.jpg
4,f75f7082d62a44e7bfd373532877c9a6,376ba3f5dfc44bfa90a4d72df1d5000f.jpg


Дальше построим ембеддинги для изображений и уже их будем кластеризовать

In [7]:
import features.cluster_builder as cluster_builder
import features.embeddings_builder as embeddings_builder
import entities
from utils.utils import (
    evaluate_model,
    create_target_dataframe
)

/home/kirill/Documents/test-interactive-standard/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Заведем параметры для CLIP и кластеризации

In [8]:
embedding_params = entities.embedding_params.EmbeddingParams(input_data_path="../data/processed/cleaned_clusters/", output_path="../models/embeddings.npy", 
                         clip_model="openai/clip-vit-large-patch14-336", device="cuda:0", allowed_extensions=[".jpg"], batch_size=32,
                          rebuild_embeddings=True)

clustering_params = entities.clustering_params.ClusteringParams(embeddings_path="../models/embeddings.npy", annoy_index_metric="angular", 
                                                               annoy_index_trees=100, linkage_method="complete", fcluster_criterion="distance",
                                                               fcluster_threshold=0.62)


In [9]:
embeddings_builder_ = embeddings_builder.EmbeddingsBuilder(embedding_params)
embeddings_builder_.build_embeddings()
images_to_pathes, all_images_ids = embeddings_builder_.get_images_to_paths()

cluster_builder_ = cluster_builder.ClusterBuilder(clustering_params)
labels, image_ids_clusters = cluster_builder_.build_image_clusters(all_images_ids)

cleaned_dataframe = pd.read_csv("../data/processed/clusters_cleaned.csv")
metrics = evaluate_model(cleaned_dataframe, create_target_dataframe(labels, images_to_pathes))

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Generating CLIP embeddings: 100%|██████████| 557/557 [01:48<00:00,  5.13it/s]


In [10]:
print(metrics)

{'homogenity_score': 0.9671600270041537, 'completeness_score': 0.9214733343408211, 'v_measure_score': 0.9437640922428762}


Вроде, получается неплохо